In [71]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from snownlp import SnowNLP
import json
import time


# 資料寫入到 JSON 檔案和 TXT 檔案
def save_to_files(results, json_file):
    with open(json_file, 'a', encoding='utf-8') as json_file:
        for result in results:
            json.dump(result, json_file, ensure_ascii=False)
            json_file.write('\n')  # 每筆資料之間插入換行符號

# 要開啟的FB mbasic網站            
url = 'https://mbasic.facebook.com'
driver = webdriver.Chrome()
driver.get(url)

# 找到網頁元素輸入帳密
email_input = driver.find_element_by_name("email")
password_input = driver.find_element_by_name("pass")

# 個人的FB帳號密碼
email_input.send_keys("ching6544239@yahoo.com")
password_input.send_keys("z4901488")

# 找到網頁的登入按鈕並點擊
login_button = driver.find_element_by_name("login")
login_button.click()

# 找到網頁的稍後再說按鈕並點擊
later_button = driver.find_element_by_link_text("稍後再說")
later_button.click()

# 找到搜尋框的元素並送出關鍵字
search_box = driver.find_element_by_name("query")
search_box.send_keys("詹記麻辣鍋")
search_box.submit()
# 等待5秒讓網頁加載完
time.sleep(5)

# 分別找到並點擊更多、貼文的按鈕
later_button = driver.find_element_by_link_text("更多")
later_button.click()

later_button = driver.find_element_by_link_text("貼文")
later_button.click()

# 循環爬取前10頁的搜索結果
for page in range(10):
    results = []  # 初始化結果列表
    try:
        # 等待所有 article 元素加載完成
        articles = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.TAG_NAME, 'article'))
        )

        # 遍歷每個文章
        for article in articles:
            # 提取 header 內容
            header = article.find_element_by_tag_name("header").text

            # 使用 XPath 找到 header 後面的第一個以 "c" 開頭的 div
            try:
                div_c = article.find_element_by_xpath('.//header/following-sibling::div[starts-with(@class, "c")]')
                p_elements = div_c.find_elements_by_tag_name('p')

                content_list = []
                for p_element in p_elements:
                    content_list.append(p_element.text)

                # 保存為 JSON 格式
                result = {"header": header, "content": content_list}
                results.append(result)
                print(json.dumps(result, ensure_ascii=False))
            except NoSuchElementException:
                print("No div with class 'c' found.")

        # 嘗試點擊"查看更多結果"鏈接
        more_results_link = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//a[contains(@href, "search/posts/") and contains(span, "查看更多結果")]'))
        )
        more_results_link.click()

        WebDriverWait(driver, 10).until(
            EC.staleness_of(articles[-1])  # 等待上一頁的最後一個 article 消失，表示新頁面已經加載完成
        )
        
        # 將本次迴圈的結果寫入檔案
        json_output_file = 'output.json'
        save_to_files(results, json_output_file)
        
        # 將 content 部分單獨保存到 txt 文件
        content_txt_output_file = 'output_content.txt'
        with open(content_txt_output_file, 'a', encoding='utf-8') as content_txt_file:
            for result in results:
                content_txt_file.write("\n".join(result["content"]) + '\n')
        
    except TimeoutException:
        print("TimeoutException: Timed out waiting for page to load.")
        driver.quit()
        break
    except Exception as e:
        print(f"An error occurred: {e}")
        break

# 關閉瀏覽器
driver.quit()


{"header": "野蠻王妃(yifenko)──在詹記麻辣火鍋 敦南店。", "content": ["久違的詹記麻辣鍋🔥", "朋友知道我太思念麻辣鍋，特別訂了詹記讓我大飽口福，實在太滿足了！", "在美國+回台隔離的日子，我最想念的就是麻辣鍋，終於等到出關能大吃，一定要先去吃鍋啊！", "這是我第二次吃詹記還是很美味，這次我們坐到包廂，更有隱密感，只是我們很烏龍，想說餐點了半小時怎麼還沒送來，原來我們根本沒送出還在桌上🤣", "詹記最有名的是鴨血，大家都狂吃鴨血，我卻愛豆腐，它的酸梅汁是跟萬波合作的，加菊花也很好喝，我們點了牛五花、牛小排，蛋餃三盤、牛筋、青菜、甜不辣、加入會員再送一盤梅花豬，都很好吃，最後甜點是一小顆冰淇淋！"]}
{"header": "野蠻王妃(yifenko)──在詹記麻辣火鍋 敦南店。", "content": ["托姐妹們的福，終於吃到詹記麻辣鍋啦\n上個月倩兒幸運訂到詹記，立馬約了這攤\n這也是我第一次吃詹記，大家都是為了鴨血而來\n但我不敢吃鴨血只敢嚐一塊，還不錯\n詹記湯頭跟肉質都蠻美味，裝潢很復古很有特色\n難怪這麼多人要來朝聖！\n聽說鼎王也不錯，下次也想吃看看", "這次順便歡迎剛回台灣的小蜜，還有特地從高雄上來的雙娜\n因為詹記有限時，吃完當然要再續攤⋯⋯"]}
{"header": "Angel安啾──在詹記麻辣火鍋 敦南店。", "content": ["聽說……我今年的母親節大餐就是【詹記麻辣鍋】\n很久沒來店裡吃\n還是好好吃❤❤❤", "Adam:週五妳有空嗎\nAngel:要幫我過母親節嗎？\nAdam:對呀！去訂詹記吧\n（明明就是老公本人最想吃詹記）"]}
{"header": "川膳麻辣鴛鴦鍋（原林口老街詹記）", "content": ["感謝客人在google評論給了五星評價並給予\n一句藏頭詩！\n川味帶勁\n膳食味美\n火辣香濃\n鍋中之王"]}
{"header": "詹記麻辣火鍋 敦南店", "content": ["就在1個月前…..疫情時期，開放內用還不能共鍋的時候…店長米奇慌慌張張訂製個人鍋專用隔板和桌上型小瓦斯爐，張張慌慌喊道：「我們要變成詹都了啦！轉型做個人鍋了啦！作戰要即時！動起來啊你各位！」", "因此美工PT小詹被要求立馬要生一大堆個人鍋內用圖，每次都是要快要便宜要有創

{"header": "尹太志", "content": ["林北一個人的詹記麻辣鍋", "無緣封頂", "淦", "-\n-"]}
{"header": "Mobile01", "content": ["詹記麻辣鍋南京店即將走入歷史，實在是令人不捨啊😫\nhttps://www.mobile01.com/newsdetail/25852/chan-chi-hot-pots-lab", "#mobile01美食版 #詹記麻辣火鍋 #招牌鴨血"]}
{"header": "芒果柚柚妹妹new life", "content": ["#關於擠尿擠便", "其實這件事大家上手以後有各自習慣的作法\n每個人也會根據狗狗的個性特性做調整", "唯一要注意的是狗狗的膀胱壁很薄\n反覆大力擠壓會破！會破！會破！\n破掉需要立即手術而且手術難度也高\n有很多飼主因為求好心切過度擠壓造成破裂的案例\n其他就看大家自己的喜好調整⋯⋯"]}
{"header": "呂柏伸", "content": ["鼻塞好幾天 食不知味\n居然連詹記麻辣鍋都覺得難吃"]}
{"header": "藤井樹", "content": ["週末小確幸\n詹記麻辣鍋\n續攤酒吧小酌！"]}
{"header": "WalkerLand 窩客島", "content": ["#今晚就吃詹記配萬波啦\n#這次聯名我喜歡", "噴汁鴨血尬烏梅湯~讚讚的！ #喵小編"]}
{"header": "愛評網", "content": ["#火鍋控們出動啦 #詹記麻辣鍋復活\n想念他們家的鴨血了\n#新家搬到這兒 #原來也是燕姿愛店\n_\n各種老鍋新鍋都在這兒👉http://bit.ly/2PpuTAu", "圖文引用自愛評 Penny愛吃吃\n「搬新家的詹記麻辣鍋，用餐前一天會發送訂位訊息到手機做確認 (如果能線上訂位就太棒了)開火加熱後麻辣鍋味道飄出來超香的啦！」"]}
{"header": "快點TV", "content": ["嗚嗚嗚，我都還沒吃過！(#克柔依)", "#詹記麻辣火鍋 #結束營業"]}
{"header": "NOWnews 今日新聞", "content": ["哦不！！！！！我最愛的鴨血ＱＱ (#助海編)\n.\n🌠加入今日LINE好友，接收最即時新聞快訊\nhttps://goo.gl/z

In [72]:
import re

def extract_chinese(text):
    # 使用正則表達式匹配中文字符
    chinese_pattern = re.compile("[\u4e00-\u9fa5]")
    result = chinese_pattern.findall(text)
    return "".join(result)

# 讀取 'output_content.txt' 文件
with open('output_content.txt', 'r', encoding='utf-8') as content_file:
    content_text = content_file.read()

# 提取中文字符
chinese_text = extract_chinese(content_text)

# 移除空格
chinese_text_no_space = chinese_text.replace(" ", "")

# 將結果寫回文件
with open('output_content_filtered.txt', 'w', encoding='utf-8') as filtered_file:
    filtered_file.write(chinese_text_no_space)


In [73]:
# 讀取已過濾的文本檔案
with open('output_content_filtered.txt', 'r', encoding='utf-8') as file:
    content = file.read()

# 使用jieba進行分詞，將結果轉為列表
seg_list = list(jieba.cut(content, cut_all=False))

# 將分詞結果轉換為字串
result = " ".join(seg_list)

# 將分詞結果寫入檔案
with open('output_segmented.txt', 'w', encoding='utf-8') as output_file:
    output_file.write(result)

# 打印分詞結果
print(seg_list)


['久', '違', '的', '詹記', '麻辣', '鍋', '朋友', '知道', '我', '太', '思念', '麻辣', '鍋特別', '訂', '了', '詹記', '讓', '我', '大', '飽', '口福', '實在', '太滿足', '了', '在', '美國', '回台', '隔離', '的', '日子', '我', '最', '想念', '的', '就是', '麻辣', '鍋終', '於', '等到', '出關', '能', '大', '吃', '一定', '要', '先去', '吃鍋', '啊', '這是', '我', '第二次', '吃', '詹記', '還是', '很', '美味', '這次', '我們', '坐', '到', '包廂', '更', '有', '隱密感', '只是', '我們', '很', '烏龍', '想', '說', '餐點', '了', '半小時', '怎麼', '還沒送', '來', '原來', '我們', '根本', '沒送', '出', '還在', '桌上', '詹記', '最', '有名', '的', '是', '鴨', '血', '大家', '都', '狂', '吃', '鴨', '血', '我', '卻', '愛', '豆腐', '它', '的', '酸梅', '汁', '是', '跟', '萬波', '合作', '的', '加', '菊花', '也', '很', '好喝', '我們', '點了', '牛', '五花', '牛小排', '蛋', '餃三盤', '牛筋', '青菜', '甜不辣', '加入', '會員', '再', '送', '一盤', '梅花', '豬都', '很', '好吃', '最後甜點', '是', '一小', '顆', '冰淇淋', '托', '姐妹', '們', '的', '福終', '於', '吃', '到', '詹記', '麻辣', '鍋啦', '上個', '月倩兒', '幸運訂', '到', '詹記', '立馬約', '了', '這攤', '這也', '是', '我', '第一次', '吃', '詹記', '大家', '都', '是', '為', '了', '鴨', '血', '而', '來', '但', '我', '不敢', '吃', '鴨', '血', '只', 

In [74]:
# 初始化情感分析结果的计数器
positive_words = []
negative_words = []

# 遍历每个词进行情感分析
for word in seg_list:
    # 使用 SnowNLP 进行情感分析
    sentiment_score = analyze_sentiment_snownlp(word)

    # 根据情感概率判断是正面还是负面，并更新对应列表
    if sentiment_score > 0.5:
        positive_words.append(word)
    else:
        negative_words.append(word)

# 计算正面和负面词汇的个数和比例
total_positive_words = len(positive_words)
total_negative_words = len(negative_words)
total_words = total_positive_words + total_negative_words

positive_ratio = total_positive_words / total_words
negative_ratio = total_negative_words / total_words

print(f"总词数: {total_words}")
print(f"正面词数: {total_positive_words}, 正面词比例: {positive_ratio:.2%}")
print(f"负面词数: {total_negative_words}, 负面词比例: {negative_ratio:.2%}")


总词数: 7358
正面词数: 4464, 正面词比例: 60.67%
负面词数: 2894, 负面词比例: 39.33%
